<a href="https://colab.research.google.com/github/liangchow/Zindi-Crop/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set Up Worksheet and Import Libraries

In [1]:
# Clone Gitub repository to Colab
from google.colab import drive
drive.mount('/content/drive')

!apt-get install git
!git clone https://github.com/liangchow/Zindi-Crop.git

# Install library
!pip install rasterio

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Cloning into 'Zindi-Crop'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 34 (delta 14), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 2.64 MiB | 9.75 MiB/s, done.
Resolving deltas: 100% (14/14), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 79.2 MB/s eta 0:00:00


In [2]:
# Import libraries
import os
import rasterio
import numpy as np
import pandas as pd

from pathlib import Path
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from rasterio.transform import from_origin

pd.set_option('display.max_columns', None)

# File Preview

In [3]:
# set data dir to Google Drive
DATA_DIR = Path('/content/drive/MyDrive/Zindi-Crop')
os.listdir(DATA_DIR)

['SampleSubmission.csv',
 'Train.csv',
 'StarterNotebook.ipynb',
 'Test.csv',
 'Slides',
 'introduction_to_remote_sensing.ipynb']

In [4]:
# Load files
train = pd.read_csv(DATA_DIR / 'Train.csv')
test = pd.read_csv(DATA_DIR / 'Test.csv')
sample_submission = pd.read_csv(DATA_DIR / 'SampleSubmission.csv')

# Preview file structures
train.shape, test.shape, sample_submission.shape

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# Preview training set
train.head()

In [ ]:
# Preview testing set
test.head()

In [ ]:
# Preview sample submission
sample_submission.head()

In [ ]:
# Check for duplicates and missing values - train
train.duplicated().any(), train.isnull().sum().any() # Handle missing values by filling them with meadian or any other techniques

In [ ]:
# Check for duplicates and missing values - test
test.duplicated().any(), test.isnull().sum().any() # Handle missing values by filling them with meadian or any other techniques

In [ ]:
# A single ID represents a unique pixel over different time periods
# create a targets_df

train_targets_df = train.groupby('ID')['Target'].first().reset_index()
train_targets_df.head()

In [ ]:
# Check data distribution
train_targets_df.Target.value_counts(normalize = True)

In [ ]:
# Aggrgate pixels over the different time periods to get a wholesome view
# Get columns to aggregate
agg_cols = train.columns.tolist()[2:-1]
agg_cols